In [37]:
# =================================================
#  Librerias Del Proyecto
# =================================================

import requests                # Simula un navegador para poder obtener todo su HTML crudo
from bs4 import BeautifulSoup  # Herramienta (clase) que convierte HTML en algo que Python pueda entender y navegar


# =====================================================
#  INICIALIZACIÓN DEL SCRAPER Y VALIDACIÓN DE CONEXIÓN
# =====================================================

url = "https://books.toscrape.com/"  # URL de la pagina pedida
response = requests.get(url)         # Creamos un objeto de respuesta de la clase requests con el comando .get() pasandole la URL
response.raise_for_status()

print(f"Estado de la respuesta de la pagina: {response.status_code}")                # 200 = todo salio bien
print(f" Pedazo del codigo HTML de la pagina:\n {response.text[:500]}\nHasta aca" )  # HTML (solo un pedazo)

# --- Creacion de la sopa de codigo ---
print("--- soup ---")
soup = BeautifulSoup(response.text, "html.parser") # sopa de datos
print(soup)

Estado de la respuesta de la pagina: 200
 Pedazo del codigo HTML de la pagina:
 <!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" /
Hasta aca
--- soup ---
<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - S

In [38]:
# =================================================
#  EXTRACCIÓN DE CATEGORÍAS DESDE EL HTML 
# =================================================

bloque_categorias = soup.find("ul", class_="nav-list").find("ul") # Busca un <ul> con la clase "nav-list"n para buscar otro <ul> dentro de el
categorias_html = bloque_categorias.find_all("a")                 # busca todos los <a> dentro del bloque de categorias (links y txt)
# (.find(un solo elemento) .find_all(lista de elementos))

# Nombre-Link
categorias = {}

# =================================================
#  SCRAPING DE CATEGORÍAS Y CONSTRUCCIÓN DE URLS
# =================================================

for c in categorias_html: 

    nombre = c.text.strip()           # Nombre (limpia) eliminando espacios. 
    href   = c["href"]                # Accede al atributo href y consigue la Url relativa.
    url_completa = url + href         # Creamos una Url completa.
    categorias[nombre] = url_completa # Agrega al diccionacio clave=nombre y valor=url_completa.


# =================================================
#  VISUALIZACIÓN DE CATEGORÍAS EXTRAÍDAS
# =================================================

print("TOTAL Categorias: ", len(categorias))
print("--- # ---")

for nombre, link in categorias.items():   # Desempaqueta la info de categoria
    print(f"Categoria: {nombre}: {link}") # Imprime el nombre y link de cada categoria

print()


TOTAL Categorias:  50
--- # ---
Categoria: Travel: https://books.toscrape.com/catalogue/category/books/travel_2/index.html
Categoria: Mystery: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Categoria: Historical Fiction: https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
Categoria: Sequential Art: https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
Categoria: Classics: https://books.toscrape.com/catalogue/category/books/classics_6/index.html
Categoria: Philosophy: https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
Categoria: Romance: https://books.toscrape.com/catalogue/category/books/romance_8/index.html
Categoria: Womens Fiction: https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html
Categoria: Fiction: https://books.toscrape.com/catalogue/category/books/fiction_10/index.html
Categoria: Childrens: https://books.toscrape.com/catalogue/category/books/

In [44]:
# Lista Global
books_data = []

# Reconstruccion pro de la URL
from urllib.parse import urljoin

# --- Diccionario Raiting ---
rating_map = {
    "One":   1,
    "Two":   2,
    "Three": 3,
    "Four":  4,
    "Five":  5
}

# -------------------------------------
# Recorremos las categorias y su URL
# -------------------------------------
for nombre_categoria, url_categoria in categorias.items():

    print("Entrando a: ", nombre_categoria)

    # Abrimos la primera pagina de la categoria
    next_page = url_categoria

    while next_page: # Mientras haya una nueva pagina entonces

        # --- Request HTTP ---
        response = requests.get(next_page)
        response.raise_for_status() # Verificador de errores

        # --- Parser Del HTMl ---
        soup     = BeautifulSoup(response.text, "html.parser")

        # --- Extraccion de los libros de la pagina ---
        books = soup.find_all("article", class_="product_pod")

        # --- Iteracion de cada libro ---
        for book in books:

            titulo = book.h3.a["title"]

            # --- Precio Limpio ---
            precio_raw    = book.find("p", class_="price_color").text
            precio_limpio = precio_raw.replace("Â", "").replace("£", "")
            precio        = float(precio_limpio)
            
            # --- Rating txt-float ---
            rating_text = book.find("p", class_="star-rating")["class"][1]
            rating = rating_map[rating_text]

            # --- Stock txt-bool ---
            stock_text  = book.find("p", class_="instock availability").text.strip()
            disponible = "In stock" in stock_text

            # --- Guardado De Datos Estructurado ---
            books_data.append({

                "titulo": titulo,
                "precio": precio,
                "rating": rating,
                "stock" : disponible,
                "categoria" : nombre_categoria
            })

        # --- Buscamos El boton De Next ---
        next_btn = soup.find("li", class_="next")

        # --- Extraemos El herf Relativo y reconstruimos la URL ---
        if next_btn:
            siguente_href = next_btn.a["href"]
            next_page = urljoin(next_page, siguente_href)
        
        else: # --- No se encontraron mas Next --- 
            next_page = None

# --- Verificaciones finales ---
print("Total libros scrapeados:", len(books_data))
type(books_data[0]["rating"])



Entrando a:  Travel
Entrando a:  Mystery
Entrando a:  Historical Fiction
Entrando a:  Sequential Art
Entrando a:  Classics
Entrando a:  Philosophy
Entrando a:  Romance
Entrando a:  Womens Fiction
Entrando a:  Fiction
Entrando a:  Childrens
Entrando a:  Religion
Entrando a:  Nonfiction
Entrando a:  Music
Entrando a:  Default
Entrando a:  Science Fiction
Entrando a:  Sports and Games
Entrando a:  Add a comment
Entrando a:  Fantasy
Entrando a:  New Adult
Entrando a:  Young Adult
Entrando a:  Science
Entrando a:  Poetry
Entrando a:  Paranormal
Entrando a:  Art
Entrando a:  Psychology
Entrando a:  Autobiography
Entrando a:  Parenting
Entrando a:  Adult Fiction
Entrando a:  Humor
Entrando a:  Horror
Entrando a:  History
Entrando a:  Food and Drink
Entrando a:  Christian Fiction
Entrando a:  Business
Entrando a:  Biography
Entrando a:  Thriller
Entrando a:  Contemporary
Entrando a:  Spirituality
Entrando a:  Academic
Entrando a:  Self Help
Entrando a:  Historical
Entrando a:  Christian
Entran

int

In [45]:
# --------------------------------------------
# TRANSFORMACION DE LA INFORMACION A DAT FRAME
# --------------------------------------------

# --- importacion de pandas ---
import pandas as pd

# --- Transformacion de la info a DataFrame ---
df = pd.DataFrame(books_data)

# --- Verificacion ---
df.info()

# --------------------------------------------
# FUNCION AUXILIAR (API Autores)
# --------------------------------------------
def obtener_autor_openlibrary(titulo):

    try:
        url_api = "https://openlibrary.org/search.json"
        params  = {"title": titulo}

        response = requests.get(url_api, params=params, timeout = 5)
        data = response.json()

        if data["docs"] and "author_name" in data["docs"][0]:
            return data["docs"][0]["author_name"][0]
        
        return None # -> Autor Desconocido
    
    except:
        return None # -> Autor Desconocido




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   titulo     1000 non-null   object 
 1   precio     1000 non-null   float64
 2   rating     1000 non-null   int64  
 3   stock      1000 non-null   bool   
 4   categoria  1000 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 32.4+ KB


In [47]:
# --------------------------------------------
# ENRIQUECIMIENTO (Agregar Autores)
# --------------------------------------------
import time # libreria de tiempo

autores = []

# --- Buscar autores de los libros ---
for i, titulo in enumerate(df["titulo"]):

    autor = obtener_autor_openlibrary(titulo)
    autores.append(autor)
    print(f"{i+1}/1000")  # progreso
    time.sleep(0.2) 

df["autor"] = autores


1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99/1000
100/1000
101/1000
102/1000
103/1000
104/1000
105/1000
106/1000
107/1000
108/1000
109/1000
110/1000
111/1000
112/1000
113/1000
114/1000
115/1000
116/1000
117/1000
118/1000
119/1000
120/1000
121/1000
122/1000
123/1000
1

In [ ]:
# -------------------------------------------
# CREACION DE BASE DE DATOS SQLITE
# -------------------------------------------
import sqlite3

# --- conectarse a la base de datos ---
conn = sqlite3.connect("books.db")

#
cursor = conn.cursor()

# =========================================
# CREACIÓN DE TABLAS (DDL)
# =========================================

cursor.execute("""
CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS autores (
    id_autor INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS libros (
    id_libro INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,
    rating INTEGER,
    stock BOOLEAN,
    id_categoria INTEGER,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor (
    id_libro INTEGER,
    id_autor INTEGER,
    FOREIGN KEY (id_libro) REFERENCES libros(id_libro),
    FOREIGN KEY (id_autor) REFERENCES autores(id_autor)
);
""")

conn.commit()